In [1]:
from sofistik_daten import *
import os                     # for the environment variable necessary
import platform               # checks the python platform
from ctypes import *          # read the functions from the cdb

# This example has been tested with Python 3.7 (64-bit)
print("The path variable=", os.environ["Path"])

# Check the python platform (32bit or 64bit)
print("Python architecture=", platform.architecture())
sofPlatform = str(platform.architecture())


if sofPlatform.find("32Bit") < 0:
    # Set environment variable for the dll files
    print ("Hint: 64bit DLLs are used")

    # Set DLL dir path - new in PY 3.8 for ctypes
    # See: https://docs.python.org/3/whatsnew/3.8.html#ctypes
    os.add_dll_directory(r"C:\Program Files\SOFiSTiK\2024\SOFiSTiK 2024\interfaces\64bit")
    os.add_dll_directory(r"C:\Program Files\SOFiSTiK\2024\SOFiSTiK 2024")

    # Get the DLL functions
    myDLL = cdll.LoadLibrary("sof_cdb_w-2024.dll")
    py_sof_cdb_get = cdll.LoadLibrary("sof_cdb_w-2024.dll").sof_cdb_get
    py_sof_cdb_get.restype = c_int

    py_sof_cdb_kenq = cdll.LoadLibrary("sof_cdb_w-2024.dll").sof_cdb_kenq_ex
else:
    # Set environment variable for the DLL files
    print ("Hint: 32bit DLLs are used")

    # Set DLL dir path - new in PY 3.8 for ctypes
    # See: https://docs.python.org/3/whatsnew/3.8.html#ctypes
    os.add_dll_directory(r"C:\Program Files\SOFiSTiK\2024\SOFiSTiK 2024\interfaces\32bit")
    os.add_dll_directory(r"C:\Program Files\SOFiSTiK\2024\SOFiSTiK 2024")

    # Get the DLL functions
    myDLL = cdll.LoadLibrary("cdb_w31.dll")
    py_sof_cdb_get = cdll.LoadLibrary("cdb_w31.dll").sof_cdb_get
    py_sof_cdb_get.restype = c_int

    py_sof_cdb_kenq = cdll.LoadLibrary("cdb_w31.dll").sof_cdb_kenq_ex


# Connect to CDB
Index = c_int()
cdbIndex = 99

# Set the CDB Path
# e.g. fileName = "S:\\test\\read_nodes.cdb"
fileName = r"C:\Users\Côme Delecourt\Desktop\TESTS\Sophistik\tuba.cdb"

# important: Unicode call!
Index.value = myDLL.sof_cdb_init(fileName.encode('utf-8'), cdbIndex)

cdbStat = c_int()  # get the CDB status
cdbStat.value = myDLL.sof_cdb_status(Index.value)

# Print the Status of the CDB
print("CDB Status:", cdbStat.value)


ie = c_int(0)
RecLen = c_int(sizeof(cnode))

nodes = []
while ie.value < 2:
   ie.value = py_sof_cdb_get(Index, 20, 0, byref(cnode), byref(RecLen), 1)
   nodes.append([
      cnode.m_nr,      # node-number
      cnode.m_xyz[0],  # x coordinates
      cnode.m_xyz[1],  # y coordinates
      ])
   
   # Always read the length of record before sof_cdb_get is called
   RecLen = c_int(sizeof(cnode))

print(nodes)
print(len(nodes))

#########################################################################################################################################

ie = c_int(0)
RecLen = c_int(sizeof(cbeam))

beams = []
while ie.value < 2:
   ie.value = py_sof_cdb_get(Index, 100, 0, byref(cbeam), byref(RecLen), 1)
   beams.append([
      cbeam.m_nr,      # node-number
      cbeam.m_node[0],
      cbeam.m_node[1],  #
      cbeam.m_dl,  # y coordinates
      cbeam.m_ldfz,
      cbeam.m_ldfy
      ])
   
   # Always read the length of record before sof_cdb_get is called
   RecLen = c_int(sizeof(cnode))

print(beams)
print(len(beams))

myDLL.sof_cdb_close(0)

# Close the CDB, 0 - will close all the files
myDLL.sof_cdb_close(0)

# Print again the status of the CDB, if status = 0 -> CDB Closed successfully
cdbStat.value = myDLL.sof_cdb_status(Index.value)
if cdbStat.value == 0:
   print("CDB closed successfully, status = 0")

The path variable= c:\Anaconda3;C:\Anaconda3;C:\Anaconda3\Library\mingw-w64\bin;C:\Anaconda3\Library\usr\bin;C:\Anaconda3\Library\bin;C:\Anaconda3\Scripts;C:\Anaconda3\bin;C:\Anaconda3\condabin;C:\windows\system32;C:\windows;C:\windows\System32\Wbem;C:\windows\System32\WindowsPowerShell\v1.0;C:\windows\System32\OpenSSH;C:\Program Files\dotnet;C:\Program Files\Git\cmd;C:\Users\Côme Delecourt\AppData\Local\Microsoft\WindowsApps;C:\Users\Côme Delecourt\AppData\Local\Programs\Microsoft VS Code\bin
Python architecture= ('64bit', 'WindowsPE')
Hint: 64bit DLLs are used
CDB Status: 3
[[1, 0.5333297252655029, -0.09796451777219772], [2, 2.411710262298584, -0.1599845290184021], [1001, 2.987422227859497, 3.0040626525878906], [1002, 2.8213515281677246, 0.39317047595977783], [1003, 4.249558925628662, 3.0040626525878906], [1004, 5.744195938110352, 2.6057913303375244], [1005, 5.744195938110352, 0.7140008211135864], [1006, 4.393486976623535, -0.10466915369033813], [1007, 3.242063522338867, -0.104669176